In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
%cd datahack-2017/
from src.models import Question, Answer, AnswerRelation
from src.main_util import load_questions

%load_ext autoreload
%autoreload 2

[Error 2] The system cannot find the file specified: u'datahack-2017/'
C:\Users\Gal\PycharmProjects\datahack-2017
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
files = ['Economics.txt', 'History_US.txt', 'Psychology.txt', 'Government.txt', 'History_World.txt']

C:\Users\Gal\PycharmProjects\datahack-2017\src


In [9]:
folder = 'MCQ/'
questions = load_questions(folder)
extract_features(questions)

100%|██████████| 5852/5852 [06:57<00:00, 14.01it/s]


In [47]:
train, tests = split_train_test(questions=questions)

In [19]:
tests[1][2].answers[0].text, tests[1][2].answers[1].text, tests[1][2].answers[2].text, tests[1][2].answers[3].text

('intrapersonal skills.', 'language.', 'creativity.', 'interpersonal skills.')

In [20]:
tests[1][1].answers[0].text, tests[1][1].answers[1].text, tests[1][1].answers[2].text, tests[1][1].answers[3].text

('language.', 'creativity.', 'interpersonal skills.', 'intrapersonal skills.')

In [38]:
from src.main_util import *
train, tests = split_train_test(questions=questions, random_state=4000000)
pd_train = build_df(train)

In [20]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples=1000, random_state=0, contamination=0, verbose=1)
pd_train = pd_train[[x for x in pd_train.columns if x not in ['a_'+str(i)+'_text' for i in range(4)] + ['q_text', 'q_category']]]
clf.fit(pd_train)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


IsolationForest(bootstrap=False, contamination=0, max_features=1.0,
        max_samples=1000, n_estimators=100, n_jobs=1, random_state=0,
        verbose=1)

In [21]:
import numpy as np
from tqdm import tqdm
res = []
for k in tqdm(range(len(tests))):
    ret = []
    for j in range(4):
        pd_test = build_df([tests[k][j]])
        pd_test
        ret.append(clf.decision_function(pd_test[[x for x in pd_train.columns if x not in ['a_'+str(i)+'_text' for i in range(4)] + ['q_text', 'q_category']]]))
    res.append(np.argmax(ret))

100%|██████████| 1463/1463 [03:13<00:00,  7.31it/s]


In [22]:
import collections
collections.Counter(res)

Counter({0: 399, 1: 370, 2: 369, 3: 325})

In [29]:
build_df([tests[0][2]]) 

,a_0_all_or_none,a_0_average_frequency,a_0_certainty_count,a_0_char_count,a_0_common_synonyms_with_question_count,a_0_common_words_with_question_count,a_0_is_correct,a_0_similarity_with_question,a_0_stopword_count,a_0_text,...,q_char_count,q_correct_answer_idx,q_is_completion,q_is_question,q_num_answers,q_stopword_count,q_text,q_title_count,q_without_stopword_count,q_word_count
0,False,14.46133,0,22,1,1,False,1.0,0,partial ego principle.,...,46,2,False,False,4,5,"According to Freud, the ego is governed by the",2,5,9


In [28]:
build_df([tests[1][2]]) 

,a_0_all_or_none,a_0_average_frequency,a_0_certainty_count,a_0_char_count,a_0_common_synonyms_with_question_count,a_0_common_words_with_question_count,a_0_is_correct,a_0_similarity_with_question,a_0_stopword_count,a_0_text,...,q_char_count,q_correct_answer_idx,q_is_completion,q_is_question,q_num_answers,q_stopword_count,q_text,q_title_count,q_without_stopword_count,q_word_count
0,False,12.044147,0,9,0,0,False,0.1,0,language.,...,78,1,False,False,4,5,Howard Gardner has theorized all of the follow...,2,7,12


In [23]:
import pandas as pd
a = pd_train.corr()

In [26]:
a[a.keys()[0]]

AttributeError: 'str' object has no attribute 'keys'

In [30]:
curr_features = list(pd_train.columns)
black_list = ['char_count', 'common_synonyms_with_question_count','common_words_with_question_count', 'is_correct', \
             'stopword_count', 'title_count', 'without_stopword_count', 'synonyms_count','correct_answer_idx', \
             'q_is_question', 'q_num_answers', 'text', 'category']
surviving_features = [x for x in curr_features if all(black not in x for black in black_list)]
print curr_features
print surviving_features


['a_0_all_or_none', 'a_0_average_frequency', 'a_0_certainty_count', 'a_0_char_count', 'a_0_common_synonyms_with_question_count', 'a_0_common_words_with_question_count', 'a_0_is_correct', 'a_0_similarity_with_question', 'a_0_stopword_count', 'a_0_title_count', 'a_0_without_stopword_count', 'a_0_word_count', 'a_1_all_or_none', 'a_1_average_frequency', 'a_1_certainty_count', 'a_1_char_count', 'a_1_common_synonyms_with_question_count', 'a_1_common_words_with_question_count', 'a_1_is_correct', 'a_1_similarity_with_question', 'a_1_stopword_count', 'a_1_title_count', 'a_1_without_stopword_count', 'a_1_word_count', 'a_2_all_or_none', 'a_2_average_frequency', 'a_2_certainty_count', 'a_2_char_count', 'a_2_common_synonyms_with_question_count', 'a_2_common_words_with_question_count', 'a_2_is_correct', 'a_2_similarity_with_question', 'a_2_stopword_count', 'a_2_title_count', 'a_2_without_stopword_count', 'a_2_word_count', 'a_3_all_or_none', 'a_3_average_frequency', 'a_3_certainty_count', 'a_3_char_c

In [54]:
from src.main_util import *
train, tests = split_train_test(questions=questions, random_state=6000000)
pd_train = build_df(train)

In [57]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples=100, random_state=0, contamination=0, verbose=1)
pd_train = pd_train[[x for x in pd_train.columns if x in surviving_features]]
clf.fit(pd_train)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished


IsolationForest(bootstrap=False, contamination=0, max_features=1.0,
        max_samples=100, n_estimators=100, n_jobs=1, random_state=0,
        verbose=1)

In [59]:
import numpy as np
from tqdm import tqdm
res = []
for k in tqdm(range(len(tests))):
    ret = []
    for j in range(4):
        pd_test = build_df([tests[k][j]])
        pd_test
        ret.append(clf.decision_function(pd_test[surviving_features]))
    res.append(np.argmax(ret))
    

100%|██████████| 1463/1463 [03:11<00:00,  7.34it/s]


In [53]:
import collections
print collections.Counter(res)
print collections.Counter(res)[0] / 1463.

Counter({0: 415, 1: 373, 3: 340, 2: 335})
0.283663704716


1

In [62]:
print collections.Counter(pd_train.a_0_all_or_none)
print collections.Counter(pd_train.a_1_all_or_none)
print collections.Counter(pd_train.a_2_all_or_none)
print collections.Counter(pd_train.a_3_all_or_none)

Counter({False: 4343, True: 46})
Counter({False: 4388, True: 1})
Counter({False: 4382, True: 7})
Counter({False: 4239, True: 150})
